In [1]:
import sys
sys.path.append("C:/Users/user/astromer")
from src.data.zero import mask_sample, get_masked, pad_inputs
import tensorflow as tf 
import warnings 
warnings.filterwarnings("ignore")

In [2]:
embedding_dimension= 256
seq_length = 200
seq_length_small = 60

## Test with padding 

mjd = tf.random.normal(shape=(seq_length,embedding_dimension))
magnitude = tf.random.normal(shape=(seq_length, embedding_dimension))
error = tf.random.normal(shape=(seq_length, embedding_dimension))

mjd_small = tf.random.normal(shape=(seq_length_small,1))
magnitude_small = tf.random.normal(shape=(seq_length_small,1))
error_small = tf.random.normal(shape=(seq_length_small,1))

print(mjd.shape, magnitude.shape, error.shape)

print(mjd_small.shape, magnitude_small.shape, error_small.shape)

(200, 256) (200, 256) (200, 256)
(60, 1) (60, 1) (60, 1)


In [38]:
input_dict = {"input": tf.squeeze(tf.stack([mjd_small,magnitude_small,error_small], -1),1)}
input_dict["mask"] = tf.ones(shape=seq_length_small)

In [39]:
msk_frac, rnd_frac, same_frac, max_obs = 0.5, 0.2, 0.2, seq_length
output = mask_sample(input_dict=input_dict,
                     msk_frac=msk_frac, 
                     rnd_frac=rnd_frac, 
                     same_frac=same_frac, 
                     max_obs=max_obs)

In [49]:
## CHECK LENGTH ### 
a = tf.shape(output["input_modified"])[0] == max_obs
b = tf.shape(output["mask_in"])[0] == max_obs
c = tf.shape(output["mask_out"])[0] == max_obs

assert (a and b and c) == True

## CHECK PADDING IN MASK_IN ##
mask_in = output["mask_in"][seq_length_small:]
test_tensor = tf.ones(shape=(max_obs -seq_length_small, tf.shape(mask_in)[-1]))
result = tf.equal(mask_in, test_tensor)
binary_result = tf.where(result, 1, 0)
assert tf.reduce_sum(binary_result) == max_obs - seq_length_small


## CHECK PADDING IN MASK_OUT##
mask_out = output["mask_out"][seq_length_small:]
test_tensor = tf.zeros(shape=(max_obs -seq_length_small, tf.shape(mask_out)[-1]))
result = tf.equal(mask_out, test_tensor)
binary_result = tf.where(result, 1, 0)
assert tf.reduce_sum(binary_result) == max_obs - seq_length_small

## CHECK PADDING IN INPUTT##
input_modified = output["input_modified"][seq_length_small:]
test_tensor = tf.zeros(shape=(max_obs -seq_length_small, tf.shape(input_modified)[-1]))
result = tf.equal(input_modified, test_tensor)
binary_result = tf.where(result, 1, 0)
assert tf.reduce_sum(binary_result) == max_obs - seq_length_small


In [49]:
#Test_Masking
frac = 0.5
mask = get_masked(magnitude, frac=frac)
assert (tf.reduce_sum(mask) / 200) == frac